# Modelo de reconocimiento de imagenes satelitales con CNN
### Santiago Fandiño Gomez
### Juan Manuel Durán 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import shutil
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns 

In [2]:
def split_dataset(source_folder, train_folder, test_folder, split_ratio=0.6):
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)
    files = os.listdir(source_folder)
    random.shuffle(files)
    num_train = int(len(files) * split_ratio)
    for file in files[:num_train]:
        src_file = os.path.join(source_folder, file)
        dst_file = os.path.join(train_folder, file)
        shutil.copy(src_file, dst_file)
    print("Carpeta de entrenamiento creada con exito")

    for file in files[num_train:]:
        src_file = os.path.join(source_folder, file)
        dst_file = os.path.join(test_folder, file)
        shutil.copy(src_file, dst_file)
    print("Carpeta de pruebas creada con exito")



In [3]:
def resize_images_in_folder(folder_name, target_size):
    
    datagen = ImageDataGenerator()
    datagen_config = {"directory": folder_name, "target_size": target_size}
    image_generator = datagen.flow_from_directory(**datagen_config, shuffle=False)
    
    for i in range(len(image_generator)):
        batch_images, batch_labels = image_generator[i]
        for j, image in enumerate(batch_images):
            if image.shape[:2] != target_size:
                resized_image = datagen.load_img(image_generator.filepaths[i * image_generator.batch_size + j], target_size=target_size)
                datagen.save_img(image_generator.filepaths[i * image_generator.batch_size + j], resized_image)

*Las dos funciones anteriores preparan las imagenes para ser trabajadas como prueba y testeo de acuerdo a los parametros que se le entreguen a las funciones. De igual manera, se trabaja una función para asignar un tamaño estandar a las imagenes de las carpetas que se especifique*

In [5]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print("GPU disponible para usar.")
else:
    print("GPU no disponible")

GPU no disponible


In [12]:
resize_images_in_folder("Data",(128,128))

Found 11262 images belonging to 12 classes.


*Teniendo todas las imagenes con el tamaño solicitado, queda realizar las divisiones de los dataset de imagenes en training y test*

In [11]:
folders = ["cloudy", "desert", "green_area", "water"]
split_ratio = 0.6

for folder in folders:
    source_folder = os.path.join("Data", folder)
    train_folder = os.path.join("Data", folder + "_train")
    test_folder = os.path.join("Data", folder + "_test")
    split_dataset(source_folder, train_folder, test_folder, split_ratio)

print("carpetas de entrenamiento y pruebas creadas")

Carpeta de entrenamiento creada con exito
Carpeta de pruebas creada con exito
Carpeta de entrenamiento creada con exito
Carpeta de pruebas creada con exito
Carpeta de entrenamiento creada con exito
Carpeta de pruebas creada con exito
Carpeta de entrenamiento creada con exito
Carpeta de pruebas creada con exito
carpetas de entrenamiento y pruebas creadas


*Teniendo las imagenes separadas en test y train se puede continuar con la definición de las redes CNN*

In [13]:
train_path_c = 'Data/cloudy_train/'
test_path_c = 'Data/cloudy_test/'
train_path_d = 'Data/desert_train/'
test_path_d = 'Data/desert_test/'
train_path_g = 'Data/green_area_train/'
test_path_g = 'Data/green_area_test/'
train_path_w = 'Data/water_train/'
test_path_w = 'Data/water_test/'


In [14]:
img_width, img_height = 128, 128
batch_size = 32

Las dimensiones anteriores son siguiendo las recomendaciones del profesor

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,      # Rotacion entre -20 y 20 grados
    width_shift_range=0.1,  # Shift horizontal hasta del 10%
    height_shift_range=0.1, # Shift vertical hasta dele 10%
    shear_range=0.2,        
    zoom_range=0.2,         # Random zoom hasta del 20%
    horizontal_flip=True,   # voltear la imagen de manera horizontal
    fill_mode='nearest'     # Llenar los nuevos pixeles creados
)

In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)